In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense

For this example, use a linear activation function within the keras library to create a regression-based neural network. 
Essentially, we are trying to predict the value of a potential car sale (i.e. how much a particular person will spend on buying a car) for a customer based on the following attributes:

-Age

-Gender

-Average miles driven per day

-Personal debt

-Monthly income

In [2]:
## Load the car.csv  to your colab instance 
from google.colab import files
uploaded = files.upload()

Saving cars.csv to cars (4).csv


In [3]:
dataset=np.loadtxt("cars.csv", delimiter=",")
x=dataset[:,0:5]
y=dataset[:,5]
y=np.reshape(y, (-1,1))
scaler = MinMaxScaler()
print(scaler.fit(x))
xscale=scaler.transform(x)

MinMaxScaler(copy=True, feature_range=(0, 1))


In [4]:
X_train, X_test, y_train, y_test = train_test_split(xscale, y)
X_train.shape

(722, 5)

# Create the model.

In [5]:
model = Sequential()
model.add(Dense(12,activation='relu',input_dim=5))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='relu'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                72        
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 9         
Total params: 185
Trainable params: 185
Non-trainable params: 0
_________________________________________________________________


# compile/fit the model

Here, we use KFold cross validation to generalize the model on unseen dataset. We use 5 folds and train the model 5 times. For each model, we calculate the loss and evalution metrics for the validation set. Then, we will choose the model with the minimum loss (MSE).


In [6]:
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_squared_error','mean_absolute_error'])
i=0
MinLoss=0
for train_index,val_index in KFold(n_splits=5).split(X_train,y_train):
  X_train2,X_val=X_train[train_index],X_train[val_index]
  y_train2,y_val=y_train[train_index],y_train[val_index]
  print(X_train2.shape)
  print(X_val.shape)
  model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_squared_error','mean_absolute_error'])
  model.fit(X_train2, y_train2, epochs=150, batch_size=1, verbose=0, validation_data=(X_val, y_val))
  loss, mse, mae = model.evaluate(X_val,y_val)
  if i==0:
      model.save("my_model.h5") 
      MinLoss=loss
  if i>0:
    if loss< MinLoss:
      MinLoss=loss
      model.save("my_model.h5")
      print("best model is",i+1)
  i=i+1

(577, 5)
(145, 5)
5/5 [==============================] - 0s 3ms/step - loss: 12850930.0000 - mean_squared_error: 12850930.0000 - mean_absolute_error: 2877.9280
(577, 5)
(145, 5)
5/5 [==============================] - 0s 3ms/step - loss: 15559574.0000 - mean_squared_error: 15559574.0000 - mean_absolute_error: 2956.6411
(578, 5)
(144, 5)
5/5 [==============================] - 0s 3ms/step - loss: 16155445.0000 - mean_squared_error: 16155445.0000 - mean_absolute_error: 2972.2891
(578, 5)
(144, 5)
5/5 [==============================] - 0s 2ms/step - loss: 11628958.0000 - mean_squared_error: 11628958.0000 - mean_absolute_error: 2430.0186
best model is 4
(578, 5)
(144, 5)
5/5 [==============================] - 0s 3ms/step - loss: 11366726.0000 - mean_squared_error: 11366726.0000 - mean_absolute_error: 2531.8625
best model is 5


In [8]:
from keras.models import load_model
model=load_model("my_model.h5")
Xnew = np.array([[40, 0, 26, 9000, 8000]])

# predict the new set of data

In [9]:
Xnewscale=scaler.transform(Xnew)
predicted = model.predict(Xnewscale)
print("X=", Xnew, ", Predicted=", predicted)

X= [[  40    0   26 9000 8000]] , Predicted= [[13233.014]]
